<a href="https://colab.research.google.com/github/maryamt95/project-files/blob/main/South_climate_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# mounting drive for data access

In [1]:
from google.colab import drive  
drive._mount('/content/drive') 

Mounted at /content/drive


# installing libraries


In [2]:
pip install cdsapi

  Created wheel for cdsapi: filename=cdsapi-0.5.1-py2.py3-none-any.whl size=11699 sha256=6e458fe9a0608f6eee64055a9bc6dab7a9a4db0ee97bf92ab59034bda2c70ad1
  Stored in directory: /root/.cache/pip/wheels/1d/2e/3c/3746e0cd076320584d6f47b688da7b13a5d5d1a92606779ca4
Successfully built cdsapi


In [1]:
import cdsapi
import netCDF4 as nc
from netCDF4 import num2date
from netCDF4 import Dataset
import numpy as np
import os
import pandas as pd

In [14]:
import xarray as xr
import os

In [15]:
south_data = xr.open_dataset('/content/drive/MyDrive/data/ERA5_monthly_south.nc')

In [16]:
south_df = south_data.to_dataframe()

In [17]:
south_df.to_csv('south_data.csv') 
!cp south_data.csv "drive/My Drive/data/"




# Read Data from Drive 

In [3]:
south_data=pd.read_csv('/content/south_data.csv')

In [4]:
south_data.head()

,expver,latitude,longitude,time,t2m,siconc,asn,smlt,sp,tco3
0,1,-66.5,-180.0,1979-01-01,273.11370,0.000000,0.879997,4.656613e-10,98775.18,0.006691
1,1,-66.5,-180.0,1979-02-01,273.69443,0.000000,0.880002,4.656613e-10,98003.59,0.006664
2,1,-66.5,-180.0,1979-03-01,272.86243,0.000000,0.879997,4.656613e-10,97982.06,0.006634
3,1,-66.5,-180.0,1979-04-01,271.37503,0.000000,0.880002,4.656613e-10,98392.62,0.006466
4,1,-66.5,-180.0,1979-05-01,266.82153,0.140235,0.880008,4.656613e-10,97862.92,0.006894


In [5]:
south_data.dtypes

expver         int64
latitude     float64
longitude    float64
time          object
t2m          float64
siconc       float64
asn          float64
smlt         float64
sp           float64
tco3         float64
dtype: object

In [6]:
len(south_data)

141177600

In [7]:
south_data.count()

expver       141177600
latitude     141177600
longitude    141177600
time         141177600
t2m           70588800
siconc        15390216
asn           70588800
smlt          70588800
sp            70588800
tco3          70588800
dtype: int64

# Rename

In [ ]:
north_data__renamed=north_data.rename(columns={'t2m':'temperature_2m','siconic':'sea_ice_area_fraction','asn':'Snow_albedo','smlt':'snow_melt','sp':'surface_pressure','tco3':
'Total_column_ozone'})

In [ ]:
north_d=north_data.drop(['expver','latitude','longitude'],axis=1)

In [ ]:
north_avg=north_d.groupby('time').mean() #keeps crushing 

In [ ]:
north_data.head(2)

,expver,latitude,longitude,time,t2m,siconc,asn,smlt,sp,tco3
0,1,90.0,-180.0,1979-01-01,244.70773,0.976897,0.879997,0.0,102661.984,0.007461
1,1,90.0,-180.0,1979-02-01,241.44653,0.987060,0.880002,0.0,101824.310,0.009453


In [ ]:
#separating columns

In [ ]:
north_data__renamed=north_d.rename(columns={'t2m':'temperature_2m','siconic':'sea_ice_area_fraction','asn':'Snow_albedo','smlt':'snow_melt','sp':'surface_pressure','tco3':
'Total_column_ozone'})

#Calculating averages 

In [3]:
south_temp=south_data[['time','t2m']]
south_temp_avg=south_temp.groupby('time').mean()

In [9]:
south_temp_avg.to_csv('Stemp.csv') 

In [ ]:
# re- indexing to eliminate the index error from sea ice 
#columns=['expver','latitude','longitude','time','t2m','siconc','asn','smlt','sp','tco3']
#north_data=north_data.reindex(columns=columns)

In [ ]:
#north_seaice=north_data[['time','siconic']]
#north_seaice_avg=north_seaice.groupby('time').mean()

In [ ]:
south_snow_albedo=south_data[['time','asn']]
south_snow_albedo_avg=south_snow_albedo.groupby('time').mean()

In [11]:
south_snow_albedo_avg.to_csv('Ssnoe_albedo.csv')

In [8]:
south_snow_melt=south_data[['time','smlt']]
south_snow_melt_avg=south_snow_melt.groupby('time').mean()

In [13]:
south_snow_melt_avg.to_csv('Ssnow_melt.csv')

In [4]:
south_spressure=south_data[['time','sp']]
south_spressure_avg=south_spressure.groupby('time').mean()

In [5]:
south_spressure_avg.to_csv('Sspressure.csv')

In [6]:
south_ozon=south_data[['time','tco3']]
south_ozon_avg=south_ozon.groupby('time').mean()

In [7]:
south_ozon_avg.to_csv('Sozon.csv')

# Merging

In [2]:
south_temp_avg=pd.read_csv('/content/Stemp.csv')
south_snow_albedo_avg=pd.read_csv('/content/Ssnoe_albedo.csv')
south_merged=pd.merge(south_temp_avg,south_snow_albedo_avg,on="time")

In [4]:
south_snow_melt_avg=pd.read_csv('/content/Ssnow_melt.csv')
south_merged_1=pd.merge(south_merged,south_snow_melt_avg,on="time")

In [5]:
south_spressure_avg=pd.read_csv('/content/Sspressure.csv')
south_merged_2=pd.merge(south_merged_1,south_spressure_avg,on="time")

In [7]:
ozon=pd.read_csv('/content/Sozon.csv')
south_data_avg=pd.merge(south_merged_2,ozon,on="time")

In [8]:

south_data_avg=south_data_avg.rename(columns={'t2m':'temperature_2m','asn':'Snow_albedo','smlt':'snow_melt','sp':'surface_pressure','tco3':'Total_column_ozone'})

In [9]:
south_data_avg.head()

,time,temperature_2m,Snow_albedo,snow_melt,surface_pressure,Total_column_ozone
0,1979-01-01,255.527956,0.856535,3.674974e-06,80884.359104,0.006313
1,1979-02-01,249.151987,0.856540,3.973923e-08,80559.107569,0.006019
2,1979-03-01,242.581619,0.856535,9.692198e-09,80029.685869,0.006080
3,1979-04-01,239.475744,0.856540,5.515169e-09,80135.939285,0.005907
4,1979-05-01,235.630476,0.856545,4.656613e-10,79953.811282,0.005999


In [10]:
south_data_avg.count()

time                  516
temperature_2m        516
Snow_albedo           516
snow_melt             516
surface_pressure      516
Total_column_ozone    516
dtype: int64

In [13]:
south_data_avg.to_csv('south_avg.csv')
!cp south_avg.csv "drive/My Drive/data/"